# ДЗ1 Анна Головина группа БКЛ213

In [59]:
from json import dumps, loads
import jsonlines
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from pymystem3 import Mystem
from string import punctuation

### 2 пункт

Поскольку в тетрадке с семинара при лемматизации пунктуация сохранялась, по аналогии здесь пунктуация тоже сохранена. Исходный файл $-$ *pokushenie.txt*. Лемматизированный текст с сохраненной пунктуацией записывается в файл *pokushenie_lemmas.txt*. Решение совровождается таймером.

In [2]:
%%time

with open("pokushenie.txt", "r", encoding="utf8") as file:
    text = file.read()
    m = Mystem()
    lemmas = m.lemmatize(text)
    with open("pokushenie_lemmas.txt", "w", encoding="utf8") as new_file:
        new_file.write("".join(lemmas))

Wall time: 52min 31s


### 3 пункт

Здесь уже исключаем пунктуацию, чтобы избежать ошибок при определении части речи. Убирала только то, что входит в строчку punctuation, поскольку она позволяет игнорировать основную массу знаков препинания, а какие-то нестандартные символы (коих меньшинство) можно вычистить из файла при необходимости уже вручную. Для токенизации берется текст из переменной, в которую уже загрузили текст файла. Запись осуществляется циклом по каждому токену, параметр ensure_ascii имеет значение False, полскольку в тексте кириллица. Чтобы не перегружать код условиями, оставила цифры (указание на годы), но в итоговом файле их частеречная принадлежность не определена (ключ *"pos"* имеет значение *"null"*).

In [8]:
%%time

morph = MorphAnalyzer()
list_of_tokens = word_tokenize(text)
with jsonlines.open("pokushenie_gram.jsonl", mode="a") as writer:
    for token in list_of_tokens:
        if token not in punctuation:
            line = {}
            info = morph.parse(token)[0]
            line["lemma"] = info.normal_form
            line["word"] = info.word
            line["pos"] = info.tag.POS
            writer.write(dumps(line, ensure_ascii=False) + "\n")

Wall time: 19.4 s


### 4 пункт

Для получения грамматической информации открываем файл на чтение и преобразуем всю полученную информацию в список. По длине списка получаем общее кол-во словоформ, т.е. знаменатель дроби при вычислении доли. Далее создаем словарь, в котором ключи $-$ названия частей речи, а значения $-$ количество словоформ, относящихся к этой части речи (None игнорируем). Я, к сожалению, не смогла придумать ничего оптимальнее, чем для получения доли от общего количества слов второй раз проходиться по ключам словаря (потому что за первый проход, т.е. собственно составление словаря значения меняются). Таким образом, получаем дроби и выводим словарь с частотностью частей речи на печать (проходясь по ключам, потому что так понятнее и симпатичнее).

In [34]:
with jsonlines.open("pokushenie_gram.jsonl", "r") as f:
    data = [token for token in f]
    quantity_gen = len(data)
    quantity_pos = {}
    for item in data:
        word = loads(item)
        if word["pos"] != None:
            quantity_pos[word["pos"]] = quantity_pos.get(word["pos"], 0) + 1
    for pos, quantity_ind in quantity_pos.items():
        print(f"Частотность {pos} равна {quantity_ind/quantity_gen}")

Частотность NOUN равна 0.25586282121202397
Частотность PREP равна 0.09752654711109686
Частотность VERB равна 0.13092297327644284
Частотность CONJ равна 0.08317025440313111
Частотность ADJF равна 0.12126656186840333
Частотность PRCL равна 0.06108241634852908
Частотность NPRO равна 0.07468480318244522
Частотность ADVB равна 0.06287895800583876
Частотность ADJS равна 0.014067562798755253
Частотность INFN равна 0.03315581790767059
Частотность GRND равна 0.0036251644156427448
Частотность PRTS равна 0.0039780565269000034
Частотность PRTF равна 0.017099226845465335
Частотность PRED равна 0.0055339899265342786
Частотность COMP равна 0.005229219466812101
Частотность INTJ равна 0.0016682172532161304
Частотность NUMR равна 0.003320393955920567


Функция здесь берет в качестве аргумента частеречный ярлык. Создается словарь, который считает частотность конкретных лемм при условии соответствия части речи ярлыку. Далее словарь сортируется по убыванию частотности. Функция возвращает список 20 самых частотных лемм-ключей. Функция далее реализуется для глаголов и наречий.

In [42]:
def top_list(label):
    pos_d = {}
    for item in data:
        word = loads(item)
        if word["pos"] == label:
            pos_d[word["lemma"]] = pos_d.get(word["lemma"], 0) + 1
        else:
            continue
    pos_sorted = {i: pos_d[i] for i in sorted(pos_d, key=pos_d.get, reverse=True)}
    res = list(pos_sorted.keys())[:20]
    return res
        
print("Топ-20 глаголов:")
print(*top_list("VERB"), sep="\n")
print("Топ-20 наречий:")
print(*top_list("ADVB"), sep="\n")

Топ-20 глаголов:
быть
мочь
стать
хотеть
знать
сказать
ждать
любить
видеть
жить
говорить
казаться
идти
савть
считать
молчать
иметь
прийтись
прийти
стоить
Топ-20 наречий:
только
ещё
уже
сейчас
тогда
теперь
уж
ничего
где
потому
тут
столь
всегда
там
никогда
много
что-то
зачем
снова
сразу


### 5 пункт

Перед применением функции уже имеющийся список токенов обновляется (убираются все токены, содержащие небуквенные символы).

Функция имеет в качестве аргумента список n-грамм. Создается пустой словарь, который заполняется n-граммами и их количеством в тексте. Далее словарь сортируется по убыванию значений (сортировка, как в предыдущих ячейках через get не сработала, поэтому была применена сортировка через лямбда-функции). Функция возвращает список 25 самых частотных n-грамм.

Применяем функцию к спискам биграмм и триграмм на основе обновленного списка токенов.

В моем случае получились именно такие (чаще со служебными частями речи) биграммы и триграммы, поскольку я не вычищала стоп-слова. Второй самой частотной группой биграмм являются имена собственные (поскольку героев часто называют по имени и отчеству). Среди триграмм второй самой частотной группой являются услоявшиеся выражения наподобие *едва ли не* и *изо всех сил*, поскольку они в целом частотны в речевом узусе носителей русского языка.

In [64]:
def frequency_count(ngrams):
    freq_dict = {}
    for ngram in ngrams:
        freq_dict[ngram] = freq_dict.get(ngram, 0) + 1
    ngram_sorted = list(sorted(freq_dict.items(), reverse = True, key=lambda x: x[1]))[:25]
    res = [' '.join(ng[0]) for ng in ngram_sorted]
    return res

new_tokens = [token for token in list_of_tokens if token.isalpha() == True]

print("Топ-25 биграмм:")
print(*frequency_count(nltk.bigrams(new_tokens)), sep = "\n")
print("Топ-25 триграмм:")
print(*frequency_count(nltk.trigrams(new_tokens)), sep = "\n")

Топ-25 биграмм:
Георгий Петрович
и не
ничего не
еще не
Толя Зыбков
не только
уже не
никогда не
у меня
Иван Трофимович
на меня
с ним
Миша Дедушка
но не
не мог
тех кто
я не
И я
вместе с
никак не
в нем
а потому
и в
в том
Ирина Михайловна
Топ-25 триграмм:
едва ли не
из тех кто
на этот раз
себя Сыном Человеческим
вместе с ним
так и не
ни у кого
о том что
в свое время
все ли равно
у кого не
друг с другом
не из тех
До сих пор
в его сторону
с ним и
И что же
направо слева направо
тем не менее
в стороне от
друг к другу
потому только что
изо всех сил
то же самое
Георгий Петрович на


### 6 пункт

Я пыталась поменять конкретные теги, не затрагивая прочую гармматическую информацию о словах, кроме того, что требуется изменить. Почему-то здесь достаточно часто слетают падежи, а в каком-никаком тексте более двух предложений отслеживание еще и падежных тегов сделает код совсем громоздким.

Местоимения не меняла, обрабатывала только существительные, прилагательные и глаголы, так что вышел не очень красивый и грамматичный текст.

In [72]:
original = """Перед вечером море Галилейское замирает: волны отбегают
от оглаженных валунов, обессиленно лежат на гальке, лишь бесшумно
вздыхают. По берегу красные, прокаленные скалы рвутся из
сочной зелени, над ними сведенные судорогой сосенки обнимаются
с жирными олеандрами. Вспыхивают весла на солнце, гонят неуклюжую
рыбачью барку. Путь недалек, в ближайший городишко Вифсаиду, он
уже виден впереди - по склону лепятся друг над другом плоские крыши
и клочковатые садики.м В барке десяток мужчин, рыбаков из Капернаума."""
rewritten = []
tokens = original.split()
morph = MorphAnalyzer()
for token in tokens:
    if (
        morph.parse(token)[0].tag.POS == "NOUN"
        or morph.parse(token)[0].tag.POS == "ADJF"
    ) and morph.parse(token)[0].tag.number == "plur":
        rewritten.append(morph.parse(token)[0].inflect({"sing"}).word)
    elif (
        morph.parse(token)[0].tag.POS == "NOUN"
        or morph.parse(token)[0].tag.POS == "ADJF"
    ) and morph.parse(token)[0].tag.number == "sing":
        rewritten.append(morph.parse(token)[0].inflect({"plur"}).word)
    elif (
        (
            morph.parse(token)[0].tag.POS == "VERB"
            or morph.parse(token)[0].tag.POS == "INFN"
        )
        and morph.parse(token)[0].tag.tense == "past"
        and morph.parse(token)[0].tag.number == "sing"
    ):
        rewritten.append(morph.parse(token)[0].inflect({"3per", "plur", "pres"}).word)
    elif (
        (
            morph.parse(token)[0].tag.POS == "VERB"
            or morph.parse(token)[0].tag.POS == "INFN"
        )
        and morph.parse(token)[0].tag.tense == "past"
        and morph.parse(token)[0].tag.number == "plur"
    ):
        rewritten.append(morph.parse(token)[0].inflect({"3per", "sing", "pres"}).word)
    else:
        rewritten.append(token)
print(" ".join(rewritten))

Перед вечером морях галилейские замирает: волн отбегают от оглаженных валунов, обессиленно лежат на гальке, лишь бесшумно вздыхают. По берега красные, прокаленные скала рвутся из сочных зелени, над ними сведенные судорогами сосенок обнимаются с жирным олеандрами. Вспыхивают весло на солнце, гонят неуклюжих рыбачьих барку. пути недалек, в ближайшие городишки Вифсаиду, он уже виден впереди - по склонам лепятся друзья над друзьями плоский крыш и клочковатый садики.мы В барках десятки мужчин, рыбака из Капернаума.
